<a href="https://colab.research.google.com/github/ab17254/ce888/blob/main/Assignment/AIBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'aif360[all]'

     |████████████████████████████████| 184kB 6.5MB/s 
     |████████████████████████████████| 2.6MB 7.5MB/s 
     |████████████████████████████████| 972kB 32.1MB/s 
     |████████████████████████████████| 21.2MB 6.4MB/s 
     |████████████████████████████████| 276kB 36.8MB/s 
     |████████████████████████████████| 9.2MB 42.5MB/s 
     |████████████████████████████████| 358kB 48.6MB/s 
     |████████████████████████████████| 21.6MB 1.6MB/s 
     |████████████████████████████████| 235kB 39.8MB/s 
     |████████████████████████████████| 3.4MB 40.5MB/s 
     |████████████████████████████████| 747kB 40.9MB/s 
     |████████████████████████████████| 25.3MB 1.5MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394769 sha256=f93a0f71f32b4581e8a457ce95a168fd33ab6fc1bff9e6ffb65839e56c0455e6
  Stored in directory: /root/.cache/pip/wheels/66/82/7b/ac2a79b8caf97e15ed415162a7f272cbba1e2e2c851fa76ae3
  Created wheel for lime: filename=lime-0.2.0.

In [ ]:
# Ensure data.zip is downloaded from github repository and uploaded to runtime
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: content/data/
  inflating: content/data/arrest_essex.csv  
  inflating: content/data/arrest_suffolk.csv  
  inflating: content/data/arrest_manchester.csv  
  inflating: content/data/arrest_lancashire.csv  
  inflating: content/data/arrest_london.csv  
  inflating: content/data/data_all.csv  
  inflating: content/data/arrest_all.csv  
  inflating: content/data/arrest_wiltshire.csv  
  inflating: content/data/arrest_all_new.csv  


In [ ]:
# Import packages
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import ReweighingMeta
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, PostProcessingMeta
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

In [ ]:
# Read data into dataframe
df = pd.read_csv('content/data/arrest_all_new.csv')

In [ ]:
# Select a random sample of 10,000 items. Anything more increases runtime and sometime causes session to crash due to high RAM usage
df = df.sample(n=10000)

In [ ]:
df['Time'] = df['Time'].str[:-3]

In [ ]:
df_copy = df.copy()

In [ ]:
# Creates an multi-index dataframe
df = df.set_index(['index','Ethnicity','Gender'])

In [ ]:
df

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
191769,All,Female,2017,All,Essex,2484
136584,Indian,Male,2010,All,Devon and Cornwall,23
143275,Any other mixed/multiple ethnic background,Female,2016,All,Staffordshire,11
1201,Mixed,Male,2006,21 years and over,West Midlands,1125
82601,Any other mixed/multiple ethnic background,Female,2010,Unknown,Surrey,-
...,...,...,...,...,...,...
207770,Mixed White and Black African,All,2014,Under 10 years,Norfolk,-
182356,Bangladeshi,All,2007,All,North Yorkshire,15
202017,Mixed White and Black African,Male,2011,18 - 20 years,Norfolk,8


In [ ]:
X = df
y = df.iloc[:,3]

In [ ]:
print(X.shape)
print(y.shape)

(10000, 4)
(10000,)


In [ ]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)

In [ ]:
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

In [ ]:
# X = pd.get_dummies(X)

In [ ]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.50)

In [ ]:
X_train

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
4656,11,2,2013,21 years and over,West Yorkshire,20
1621,13,1,2014,All,Staffordshire,65
7346,4,0,2011,Under 10 years,Merseyside,-
2855,2,2,2007,18 - 20 years,Devon and Cornwall,13
1543,8,1,2011,All,Thames Valley,368
...,...,...,...,...,...,...
3125,4,1,2008,18 - 20 years,Gloucestershire,2
6987,12,0,2008,18 - 20 years,Northumbria,26
9514,14,1,2006,Under 10 years,Essex,-


In [ ]:
X_test

,,,Time,Age_Group,Geography,Number of arrests
index,Ethnicity,Gender,,,,
4902,10,1,2014,18 - 20 years,Dyfed-Powys,2
7494,4,0,2012,10 - 17 years,North Wales,5
3650,22,1,2009,Unknown,Metropolitan Police,1
5443,10,1,2016,18 - 20 years,Kent,6
4598,7,2,2013,Under 10 years,Bedfordshire,-
...,...,...,...,...,...,...
9486,15,0,2011,All,Warwickshire,26
3834,12,1,2010,10 - 17 years,Thames Valley,7
2625,4,1,2006,10 - 17 years,Devon and Cornwall,4


In [ ]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(X_train)
X_train  = pd.DataFrame(ohe.transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), index=X_test.index)

In [ ]:
y_train.head()

index  Ethnicity  Gender
4656   11         2          455
1621   13         1         1245
7346   4          0            0
2855   2          2          181
1543   8          1          855
dtype: int64

In [ ]:
X_train

,,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994
index,Ethnicity,Gender,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4656,11,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1621,13,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7346,4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2855,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1543,8,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,4,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6987,12,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9514,14,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Initial run of algorithm
y_pred = LogisticRegression(solver='lbfgs').fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

0.807

In [ ]:
disparate_impact_ratio(y_test, y_pred, prot_attr='Ethnicity')

1.0125786163522013

In [ ]:
# Debiased and hypertuned run of algorithm
rew = ReweighingMeta(estimator=LogisticRegression(solver='lbfgs'))

params = {'estimator__C': [1, 10], 'reweigher__prot_attr': ['Ethnicity']}

clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8018
{'estimator__C': 10, 'reweigher__prot_attr': 'Ethnicity'}


In [ ]:
disparate_impact_ratio(y_test, clf.predict(X_test), prot_attr='Ethnicity')


1.0447239692522712